En este archivo puedes escribir lo que estimes conveniente. Te recomendamos detallar tu solución y todas las suposiciones que estás considerando. Aquí puedes ejecutar las funciones que definiste en los otros archivos de la carpeta src, medir el tiempo, memoria, etc.

Lo primero que quiero revisar es si el archivo existe.

In [1]:
import os

file_path = r"\Users\DZWorld2\Documents\Studying\challenge_option\tweets.json\farmers-protest-tweets-2021-2-4.json"
# Ensure the file path is correctly formatted for Windows
file_path = file_path.replace("\\", "/")

# Check if the file exists
if os.path.exists(file_path):
    print("The file exists.")
else:
    
    print("The file does not exist.")

The file exists.


Se importaran las soluciones implementas las cuales se encuetnran dentro de src. para facilitar la lectura.

In [2]:
from src import q1_memory, q1_time, q2_memory, q2_time, q3_memory, q3_time

In [3]:
# Para tener una mejor visualización en jupyter usare pandas
import pandas as pd

In [26]:
q1_memory_data = q1_memory.q1_memory(file_path)
df_q1_memory = pd.DataFrame(q1_memory_data,columns=['Date', 'Username', 'Count'] )
df_q1_memory

,Date,Username,Count
0,2021-02-12,RanbirS00614606,176
1,2021-02-13,MaanDee08215437,178
2,2021-02-14,rebelpacifist,119
3,2021-02-15,jot__b,134
4,2021-02-16,jot__b,133
5,2021-02-17,RaaJVinderkaur,185
6,2021-02-18,neetuanjle_nitu,195
7,2021-02-19,Preetm91,267
8,2021-02-20,MangalJ23056160,108
9,2021-02-23,Surrypuria,135


In [4]:
%load_ext memory_profiler

In [28]:
%memit q1_memory.q1_memory(file_path)

peak memory: 125.16 MiB, increment: 1.58 MiB


In [29]:
%timeit q1_memory.q1_memory(file_path)

4.95 s ± 146 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
q1_time_data_s = q1_time.q1_time(file_path)
df_q1_time_s = pd.DataFrame(q1_time_data_s,columns=['Date', 'Username'] )
df_q1_time_s 

,Date,Username
0,2021-02-12,RanbirS00614606
1,2021-02-13,MaanDee08215437
2,2021-02-17,rebelpacifist
3,2021-02-16,jot__b
4,2021-02-14,jot__b
5,2021-02-18,RaaJVinderkaur
6,2021-02-15,neetuanjle_nitu
7,2021-02-20,Preetm91
8,2021-02-23,MangalJ23056160
9,2021-02-19,Surrypuria


Para Q1_Time implementare Pyspark directamente en este notebook con el fin de tener una solución time efficient y escalable considerando que par aeste ejercicio trabajaremos por fechas con muchos usuarios por fecha siendo asi una posible partición perfecta para pyspark y su manera distrbuida de trabajar. 

In [4]:
from typing import List, Tuple
from datetime import date
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, count, row_number
from pyspark.sql.window import Window

def q1_time_spark(file_path: str) -> List[Tuple[date, str, int]]:
  try:
    # Initialize Spark session
    spark = SparkSession.builder.appName("Q1Time").getOrCreate()

    # Read JSON file
    df = spark.read.json(file_path)

    # Filter out rows with missing fields
    df_filtered = df.filter("date IS NOT NULL AND user IS NOT NULL AND id IS NOT NULL AND user.username IS NOT NULL")

    # Select and rename fields
    df_selected = df_filtered.select(to_date(col("date")).alias("date"), col("user.username").alias("user"), col("id"))

    # Count tweets per date and get top 10 dates
    top_dates_df = df_selected.groupBy("date").count().orderBy(col("count").desc()).limit(10)

    # Join back to get tweets from top 10 dates only
    df_top_dates = df_selected.join(top_dates_df.select("date"), "date")

    # Get top user per date
    windowSpec = Window.partitionBy("date").orderBy(col("count").desc())
    top_users_df = df_top_dates.groupBy("date", "user").agg(count("id").alias("count")).withColumn("row_number", row_number().over(windowSpec)).filter(col("row_number") == 1).select("date", "user", "count")

    # Collect result to driver with tweet count
    result = [(row.date, row.user, row['count']) for row in top_users_df.collect()]

    spark.stop()
    return result
  except Exception as e:
    print(f"An unexpected error occurred: {e}")
    return []

In [6]:
q1_time_data = q1_time_spark(file_path)
df_q1_time = pd.DataFrame(q1_time_data,columns=['Date', 'Username', 'Count'] )
df_q1_time

,Date,Username,Count
0,2021-02-12,RanbirS00614606,176
1,2021-02-13,MaanDee08215437,178
2,2021-02-14,rebelpacifist,119
3,2021-02-15,jot__b,134
4,2021-02-16,jot__b,133
5,2021-02-17,RaaJVinderkaur,185
6,2021-02-18,neetuanjle_nitu,195
7,2021-02-19,Preetm91,267
8,2021-02-20,MangalJ23056160,108
9,2021-02-23,Surrypuria,135


Como se puede ver usando Pyspark obtener una mejora sustantiva que se notaría mas si tuvieramos una mayor cantidad de datos.

In [8]:
%timeit q1_time_spark(file_path)

3.19 s ± 329 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [8]:
%timeit q1_time.q1_time(file_path)

4.47 s ± 146 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


Para Q2 debido a que se quiere mostrar los emojis directamente genera posibles delay a la hora de obtener estos. Por lo que se buscara una solución con función generadora para ir linea por linea ya que de esta es una excelente solución para la eficiencia de la memoria. 

In [17]:
q2_memory_data = q2_memory.q2_memory(file_path)
df_q2_memory = pd.DataFrame(q2_memory_data,columns=['Emoji', 'count'] )
q2_time_data = q2_time.q2_time(file_path)
df_q2_time = pd.DataFrame(q2_time_data,columns=['Emoji', 'count'] )

In [18]:
df_q2_memory

,Emoji,count
0,🙏,5049
1,😂,3072
2,🚜,2972
3,🌾,2182
4,🇮🇳,2086
5,🤣,1668
6,✊,1651
7,❤️,1382
8,🙏🏻,1317
9,💚,1040


In [19]:
df_q2_time

,Emoji,count
0,🙏,5049
1,😂,3072
2,🚜,2972
3,🌾,2182
4,🇮🇳,2086
5,🤣,1668
6,✊,1651
7,❤️,1382
8,🙏🏻,1317
9,💚,1040


Como podemos observar la solución que utiliza funciones generadoras "q2_memory" es mas eficiente en el uso de la memoria ya que de esta manera logramos la lazy evaluation debido a que solo obtendremos el dato cuando es llamada y no se cargar el dataset directo en la memoria lo cual sería ineficiente y mas en caso donde se maneje grandes volumenes de data.
Para este caso es dificil encontrar una solución time efficient debido a que es necesario usar la librería emoji para visualizar estos un posible approach para esta sería tener guardados los emojis en un diccionario hasheable para facilitar traer estos.

In [10]:
%memit q2_memory.q2_memory(file_path)

peak memory: 103.75 MiB, increment: 0.96 MiB


In [12]:
%timeit q2_memory.q2_memory(file_path)

13.2 s ± 332 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%memit q2_time.q2_time(file_path)

peak memory: 150.54 MiB, increment: 37.29 MiB


In [22]:
%timeit q2_time.q2_time(file_path)

13.4 s ± 177 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [26]:
q3_memory_data = q3_memory.q3_memory(file_path)
df_q3_memory = pd.DataFrame(q3_memory_data,columns=['UserName', 'count'] )
q3_time_data = q3_time.q3_time(file_path)
df_q3_time = pd.DataFrame(q3_time_data,columns=['UserName', 'count'] )

In [27]:
df_q3_memory

,UserName,count
0,narendramodi,2261
1,Kisanektamorcha,1836
2,RakeshTikaitBKU,1639
3,PMOIndia,1422
4,RahulGandhi,1125
5,GretaThunberg,1046
6,RaviSinghKA,1015
7,rihanna,972
8,UNHumanRights,962
9,meenaharris,925


In [28]:
df_q3_time

,UserName,count
0,narendramodi,2261
1,Kisanektamorcha,1836
2,RakeshTikaitBKU,1639
3,PMOIndia,1422
4,RahulGandhi,1125
5,GretaThunberg,1046
6,RaviSinghKA,1015
7,rihanna,972
8,UNHumanRights,962
9,meenaharris,925


En este caso donde se realiza una consulta relativamente simple podemos observar que no difieren mucho los resultado esto es debido al grado de complejidad y al tamaño del set de datos. En estos casos conviene encontrar una solución sencilla ya que no es necesario usar herramientas mas complejas como puede llegar a ser spark.

In [29]:
%memit q3_memory.q3_memory(file_path)

peak memory: 119.25 MiB, increment: 0.29 MiB


In [30]:
%timeit q3_memory.q3_memory(file_path)

3.45 s ± 144 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [31]:
%memit q3_time.q3_time(file_path)

peak memory: 119.49 MiB, increment: 0.02 MiB


In [32]:
%timeit q3_time.q3_time(file_path)

3.34 s ± 94.3 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
